### Warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### ML Client session

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

registry_name = 'azureml'
credential = DefaultAzureCredential()
ml_client =  MLClient.from_config(credential=credential)

ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name
)